In [1]:
import pandas as pd
import plotly.express as px

### I. Importing Raw Data

In [2]:
data =  pd.read_excel("Umfrage_unbereinigt.xlsx")

In [3]:
df_snacks = pd.DataFrame()

df_snacks["orders"]= data["Welche Snacks dürfen für dich bei einem Filmabend mit Freunden nicht fehlen?"]

df_snacks.shape


(190, 1)

In [4]:
df_snacks.head()

,orders
0,"Chips, Popcorn (süß), Erdnüsse, M&Ms"
1,Nachos (opt. mit Käsesauce und/oder Guacamole ...
2,"Chips, Popcorn (süß), Nachos (opt. mit Käsesau..."
3,"Chips, Nachos (opt. mit Käsesauce und/oder Gua..."
4,"Chips, Nachos (opt. mit Käsesauce und/oder Gua..."


In [5]:
#transforming orders column so each contains a list

df_snacks["orders"] = df_snacks["orders"].str.split(", ")

df_snacks.head()

,orders
0,"[Chips, Popcorn (süß), Erdnüsse, M&Ms]"
1,[Nachos (opt. mit Käsesauce und/oder Guacamole...
2,"[Chips, Popcorn (süß), Nachos (opt. mit Käsesa..."
3,"[Chips, Nachos (opt. mit Käsesauce und/oder Gu..."
4,"[Chips, Nachos (opt. mit Käsesauce und/oder Gu..."


In [6]:
#replace all instances of "Nachos (opt. mit Käsesauce und/oder Guacamole und/oder Salsa)" with "Nachos"

df_snacks["orders"] = df_snacks["orders"].apply(lambda x: ["Nachos" if i == "Nachos (opt. mit Käsesauce und/oder Guacamole und/oder Salsa)" else i for i in x])

### II. Frequency Analysis

In [7]:
#analyse the popularity of each snack

from collections import Counter

count = Counter()

for row in df_snacks["orders"]:
    count.update(row)

count.most_common(10)

[('Chips', 123),
 ('Nachos', 72),
 ('Popcorn (süß)', 55),
 ('Schokolade', 51),
 ('Gummibärchen', 47),
 ('Erdnüsse', 40),
 ('M&Ms', 37),
 ('Kekse', 37),
 ('Eiscreme', 31),
 ('Popcorn (salzig)', 20)]

In [8]:
count.most_common()[-10:]

[('Kichererbsen', 1),
 ('Käse-Cracker', 1),
 ('Saure Gummibaecht', 1),
 ('Flips', 1),
 ('Softdrinks', 1),
 ('Knabberbox', 1),
 ('Ich mag keine Snacks', 1),
 ('Toffiffee', 1),
 ('Keine ', 1),
 ('Coca Cola', 1)]

In [9]:
df_counted = pd.DataFrame(count.most_common(), columns=["snack", "count"])

In [17]:
fig = px.treemap(df_counted, path=["snack"], values="count", title="Snacks bei Filmabenden")

fig.show()

### III. Creating datasets for A priori & FP-Growth

In [14]:
#create dummy columns for each snack

df_snacks_dummies = df_snacks["orders"].explode().str.get_dummies()

df_snacks_dummies = df_snacks_dummies.groupby(df_snacks_dummies.index).sum()


In [15]:
#export the data so we can use them in other notebooks#

df_snacks_dummies.to_csv("snacks.csv", index=False)

### IV. Creating dataset for Eclat

In [29]:
df_snacks_eclat = pd.DataFrame([pd.Series(x) for x in df_snacks["orders"]])
df_snacks_eclat.columns = ['{}'.format(x) for x in df_snacks_eclat.columns]

df_snacks_eclat.head()

,0,1,2,3,4,5,6,7,8
0,Chips,Popcorn (süß),Erdnüsse,M&Ms,NaN,NaN,NaN,NaN,NaN
1,Nachos,M&Ms,Lecker Bierchen,NaN,NaN,NaN,NaN,NaN,NaN
2,Chips,Popcorn (süß),Nachos,M&Ms,NaN,NaN,NaN,NaN,NaN
3,Chips,Nachos,Schokolade,Erdnüsse,Kekse,M&Ms,Eiscreme,Bier,NaN
4,Chips,Nachos,Kekse,Salzstangen,NaN,NaN,NaN,NaN,NaN


In [30]:
df_snacks_eclat.to_csv("snacks_eclat.csv", index=False)